In [47]:
import numpy as np
import scipy.stats as scps
from scipy.special.orthogonal import ps_roots
import scipy.interpolate as scin
from copy import *
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from ret import (
    util,
    egm_step
)
from aux import create_container

In [49]:
import pickle
with open('m0_value.pkl', 'rb') as file : 
    m0_value = pickle.load(file)

with open('m0_policy.pkl', 'rb') as file : 
    m0_policy = pickle.load(file)


# Model parameters (default)

In [85]:
# Number of periods (fist period is t=1) 
Tbar = 25

# Number of grid points over assets
ngridm = 500

# Maximum level of assets
mmax = 50

# Number of quadrature points used in calculation of expectations
n_quad_points = 5

# Number of simulations
nsims = 50

# Interval of the initial wealth
init = [10, 30]

# Interest rate
r = 0.05

# Discount factor
beta = 0.95

# Standard deviation of log-normally distributed income shocks
sigma = 0.00

# Disutility of work
duw = 0.35

# CRRA coefficient (log utility if ==1)
theta = 1.95

# Careful with the coefficients here -- original code had the polynomial
# Coded as a + b * x - c * x**2 ... (note the crazy minus)
coeffs_age_poly = np.array([0.75, 0.04, -0.0002])

# Consumption floor (safety net in retirement)
cfloor = 0.001

# Scale of the EV taste shocks 
lambda_ = 2.2204e-16

In [86]:
# Initialize grids
quadstnorm = scps.norm.ppf(ps_roots(n_quad_points)[0])
quadw = ps_roots(n_quad_points)[1]

In [87]:
savingsgrid = np.linspace(0, mmax, ngridm)

In [88]:
policy, value = create_container(ngridm, Tbar, mmax, theta, duw)

In [89]:
for period in [23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]:
    value, policy = egm_step(value, policy, savingsgrid, quadstnorm, period, Tbar, ngridm, cfloor, n_quad_points, r, coeffs_age_poly, theta, duw, beta, lambda_, sigma, quadw)

In [90]:
for point in [23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9,  8, 7, 6, 5, 4, 3, 2, 1, 0]:
    np.testing.assert_almost_equal(m0_value[:, 0, 1, point].T[~np.isnan(m0_value[:, 0, 1, point])], value[point][1].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_value[:, 1, 1, point].T[~np.isnan(m0_value[:, 1, 1, point])], value[point][1].T[1], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 0, 1, point].T[~np.isnan(m0_policy[:, 0, 1, point])], policy[point][1].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 1, 1, point].T[~np.isnan(m0_policy[:, 1, 1, point])], policy[point][1].T[1], decimal=8)
    np.testing.assert_almost_equal(m0_value[:, 0, 0, point].T[~np.isnan(m0_value[:, 0, 0, point])], value[point][0].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_value[:, 1, 0, point].T[~np.isnan(m0_value[:, 1, 0, point])], value[point][0].T[1], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 0, 0, point].T[~np.isnan(m0_policy[:, 0, 0, point])], policy[point][0].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 1, 0, point].T[~np.isnan(m0_policy[:, 1, 0, point])], policy[point][0].T[1], decimal=8)


In [91]:
from dcegm.solve.solve import solve_retirement_model

In [92]:
value_, policy_ = solve_retirement_model(ngridm, n_quad_points, r, coeffs_age_poly, theta, duw, beta, lambda_, sigma, mmax, Tbar)

[[array([[ 0.00000000e+00, -1.05373763e+04],
         [ 1.00128448e-03, -1.07537527e+04],
         [ 1.07874813e-01, -4.33086936e+03],
         ...,
         [ 5.34561165e+01,  1.07222777e+01],
         [ 5.35636741e+01,  1.07308432e+01],
         [ 5.36712316e+01,  1.07393752e+01]]),
  array([[ 0.        ,  7.44053306],
         [ 4.49553039,  7.51871174],
         [ 4.70077341,  7.52941583],
         [ 4.80833099,  7.53489198],
         [ 4.91588857,  7.54035135],
         [ 5.02344615,  7.54579408],
         [ 5.13100373,  7.55122019],
         [ 5.2385613 ,  7.55662971],
         [ 5.34611888,  7.56202255],
         [ 5.45367646,  7.56739856],
         [ 5.56123404,  7.57275765],
         [ 5.66879162,  7.57809986],
         [ 5.7763492 ,  7.58342534],
         [ 5.88390678,  7.58873431],
         [ 5.99146435,  7.59402699],
         [ 6.09902193,  7.59930358],
         [ 6.20657951,  7.60456426],
         [ 6.31413709,  7.60980922],
         [ 6.42169467,  7.61503861],
         [ 